In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

In [5]:
def read_data_from_subjects(index):
    dataset = []
    folder = '../data/subjects-xy/'
    files = os.listdir(folder)

    for f in files:
        data = pd.read_csv(folder + f, delimiter=";").to_numpy()
        positions = data[:,index]
        dataset.append(positions)

    return dataset

In [7]:
#Full data acquired here:
dataset = pd.read_csv('../data/full_data.csv')

In [28]:
gaze_x = dataset['Gaze.x.'] / 2560
gaze_y = dataset['Gaze.y.'] / 1080

In [35]:
positions = read_data_from_subjects([2, 4])
gaze_x = read_data_from_subjects(-2)
gaze_y = read_data_from_subjects(-1)

gaze_x[1]
gaze_y[1]

max(gaze_x[0])

358954.2

In [ ]:
def read_data_from_set():
    